# LangChain for LLM(Palm2) Application Development
## https://github.com/hwchase17/langchain/pull/3575

In [1]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file

# configure palm
import google.generativeai as palm
api_key = os.environ["GOOGLE_API_KEY"] # 
palm.configure(api_key=api_key)


In [2]:
response = palm.generate_text(prompt="The opposite of hot is")
print(response.result)  # cold.

cold.


In [4]:
from langchain.embeddings import GooglePalmEmbeddings

embeddings = GooglePalmEmbeddings()

# Generate embeddings for a given text

text = "This is a sample text."
embeddings = embeddings.embed_query(text)

# Print the embeddings
print(embeddings)

[0.0061349943, 0.015134675, 0.0012696368, 0.007476202, 0.008996238, -0.00922537, -0.016495135, 0.0016385056, -0.00095049915, -0.03672091, 0.02080144, -0.015795728, 0.018596753, 0.0050777276, -0.018418409, -0.03107394, -0.0393124, 0.0054978863, 0.047986586, 0.014276307, -0.07606485, -0.01134721, -0.013535012, -0.010288683, 0.0040683714, -0.07637619, 0.043953948, 0.04945843, -0.040389434, -0.033197567, -0.014906142, -0.003327762, -0.022006249, 0.015078855, 0.018143233, -0.032327548, -0.0018663215, 0.036788303, 0.03930756, 0.042413667, -0.017809885, -0.012181523, 0.045432754, -0.014605079, -0.012039571, -0.00024533994, -0.037588913, -0.026974002, -0.009648986, -0.042725068, -0.032784533, -0.0052300612, -0.017744195, 0.026238488, -0.04278566, 0.014807797, -0.06545243, 0.0144396, -0.036777977, -0.038399287, 0.016613444, 0.018550593, 0.03870792, -0.075784944, -0.05402422, 0.04311161, 0.03465176, 0.0007594819, -0.016252764, -0.021226626, 0.0063584656, 0.024489375, -0.046267968, -0.018162046, 

In [5]:
from langchain.llms import GooglePalm
llm = GooglePalm()
llm.temperature = 0.1

prompts = ["The opposite of hot is",'The opposite of cold is'] # according to the class prmpts must be in list
llm_result = llm._generate(prompts)

In [6]:
llm_result.generations[0][0].text
llm_result.generations[1][0].text


'hot'

# Why are the results stored in a **multidimensional array?**

Imagine you are working with a bard. 

- You provide **1** input to the bard. 
- The bard processes it and returns **3** outputs. 

Similarly, in this case, we can obtain a maximum of **8** outputs. 

- The first dimension represents each prompt in the prompts list.
- The second dimension represents the answers for each input. 


# let's make simple QA from websites

In [7]:
from langchain.document_loaders import UnstructuredURLLoader  #load urls into docoument-loader
from langchain.chains.question_answering import load_qa_chain
from langchain.indexes import VectorstoreIndexCreator #vectorize db index with chromadb
from langchain.text_splitter import CharacterTextSplitter #text splitter
from langchain.chains import RetrievalQA


In [8]:
urls = ['https://www.linkedin.com/pulse/transformers-without-pain-ibrahim-sobh-phd/',]
loader = [UnstructuredURLLoader(urls=urls)]
index = VectorstoreIndexCreator(
        embedding=GooglePalmEmbeddings(),
        text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loader)

In [9]:
chain = RetrievalQA.from_chain_type(llm=llm, 
                            chain_type="stuff", 
                            retriever=index.vectorstore.as_retriever(), 
                            input_key="question")

In [10]:
answer = chain.run('What is machine translation?')
answer

'Machine Translation (MT) is the task of translating a sentence x from one language (the source language) to a sentence y in another language (the target language).'